In [32]:
import csv
import torch
from tqdm import tqdm
from model import Net
from verifier import ABCrown
from torchvision import datasets
from torchvision.transforms import ToTensor

In [33]:
model_weigths_file = "./ex_model.pt"
log_file_name = "try.csv"
n_images = 2   # number of images to test
epsilon_list = torch.linspace(0, 0.5, 5)
torch.manual_seed(0)

In [34]:
# loading the model
n_classes = 10
image_dimension = (3, 32, 32)
net = Net(image_dimension[0], n_classes)
net.load_state_dict(torch.load(model_weigths_file))
net = net.eval()

In [35]:
# getting the images
images_dataset = datasets.CIFAR10(root="data", train=False, download=True, transform=ToTensor())
image_indexes = torch.randint(0, len(images_dataset), (n_images,), dtype=torch.int)
selected_images = ([images_dataset[i] for i in image_indexes])  # type: ignore

In [36]:
# setting the verifier
verifier = ABCrown(device='cpu')

In [ ]:
# opening the csv
with open(log_file_name, mode="w", newline="") as file:
    # writing the header
    writer = csv.DictWriter(file, fieldnames=['image_id', 'eps', 'status', 'success'])
    writer.writeheader()
    adversarial_examples = []
    # iterating through the images
    for i, (image, label) in tqdm(enumerate(selected_images)):
        adversarial_example_per_image = []
        # iterating thtough the epsilons
        for eps in epsilon_list:
            # if there is an error, don't crash
            try:
                result = verifier.verify(net, image, n_classes, label, eps.item()).as_dict()
            except:
                # save a dummy tensor (all -1s)
                print(f"> Error at image {i} and eps={eps}")
                info_to_save = {
                    "image_id": i,
                    "eps": eps.item(),
                    "status": "CRASH",
                    "success": False,
                }
                adversarial_example_per_image.append(-torch.ones_like(image))
                writer.writerow(info_to_save)
                file.flush()
                pass
            
            # saving result
            info_to_save = {
                "image_id": i,
                "eps": eps.item(),
                "status": result['status'],
                "success": result['success'],
            }
            writer.writerow(info_to_save)
            file.flush()
            
            # saving adversarial example
            if result['stats']['attack_examples'].shape[0] != 0:
                adversarial_example_per_image.append(result['stats']['attack_examples'][0])
            else:
                adversarial_example_per_image.append(-torch.ones_like(image))
                
        adversarial_examples.append(torch.stack(adversarial_example_per_image))

# saving the obtained adversarial examples
torch.save(torch.stack(adversarial_examples), "adversarial_examples.pt")

2it [00:00, 284.74it/s]

> Error at image 0 and eps=0.0
> Error at image 0 and eps=0.125
> Error at image 0 and eps=0.25
> Error at image 0 and eps=0.375
> Error at image 0 and eps=0.5
> Error at image 1 and eps=0.0
> Error at image 1 and eps=0.125
> Error at image 1 and eps=0.25
> Error at image 1 and eps=0.375
> Error at image 1 and eps=0.5
